In [5]:
import pandas as pd
import numpy as np
from numpy.linalg import eig, eigh, eigvalsh, norm
from scipy.stats import multivariate_normal, invwishart, invgamma

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
np.random.seed(324)
J = 6
K = 2
beta = np.array([[1,0], [-2, 1],[-1,-1],[2,-1], [3,-1], [1,-1]], dtype=float)


Construct the matrix $M = \beta \beta^T$ which is real and symmetric. 
Let $dim(M)= (n \times n)$ and rank$(M)=k$ with $k<m$.  
 
Here are some facts about real symmetric matrices

* all eigenavalues are real
* eigenvectors belonging to distinct eigenvalues are orthogonal
* rank( $\beta$ ) = rank( $\beta \beta^T$ ) = rank( M ) = $k$
* $M$ has exactly $k$ non-zero eigenvalues
* if $(\lambda_1\dots \lambda_n)$ are the eigenvalues (not necessarily distinct), there exist $n$ orthonormal eigenvectors $v_1 \dots v_n$ that correspond respectively to $(\lambda_1 \dots \lambda_n)$


* are all eigenvalues positive?

In [7]:
M = beta@beta.T; M

array([[ 1., -2., -1.,  2.,  3.,  1.],
       [-2.,  5.,  1., -5., -7., -3.],
       [-1.,  1.,  2., -1., -2.,  0.],
       [ 2., -5., -1.,  5.,  7.,  3.],
       [ 3., -7., -2.,  7., 10.,  4.],
       [ 1., -3.,  0.,  3.,  4.,  2.]])

In [8]:
eset = eigh(M)
L = eset[0] ; L

array([-4.03328716e-15, -7.37570285e-16,  1.35256974e-17,  1.11441244e-15,
        2.24085774e+00,  2.27591423e+01])

In [9]:
P = eset[1] ; P

array([[-0.94476253, -0.08456539, -0.04426746, -0.01654158,  0.24496801,
        -0.19501253],
       [-0.12531505,  0.5261797 ,  0.08574234,  0.68175156,  0.1315528 ,
         0.46689183],
       [-0.26514477,  0.31263616, -0.18638773, -0.180307  , -0.86645684,
         0.11814577],
       [-0.00339831,  0.30257632,  0.82041522,  0.00779524, -0.1315528 ,
        -0.46689183],
       [ 0.14627813,  0.46672055, -0.53159045,  0.16494195,  0.11341521,
        -0.66190436],
       [-0.0030501 , -0.55575332, -0.0166947 ,  0.68932138, -0.37652082,
        -0.27187929]])

In [10]:
# eigenequation is satisfied
for i in range(6):
    l = L[i]
    v = P[:,i]
    c = M@v - l*v
    print(np.allclose(c, np.zeros(6)))


True
True
True
True
True
True


In [11]:
# all eigenvectors have norm 1
norm(P, axis=1)

array([1., 1., 1., 1., 1., 1.])

In [12]:
# matrix P of eigenvectors satisfies PP' = I , i.e. P' = P^-1
np.allclose(P@P.T, np.eye(6))

True

In [13]:
# Check that M = P L P' = P L P^-1
np.allclose(P @ np.diag(L) @ P.T, M)


True

In [14]:
# Check that L = P' M P = P^-1 M P
np.allclose(P.T @ M @ P, np.diag(L) )

True

We know that $M$ should have 3 zero eigenvalues. We set all the small values of `L` to zero and see if we recover the same results

In [15]:
Lt = L.copy() 
Lt[:4] = 0; Lt


array([ 0.        ,  0.        ,  0.        ,  0.        ,  2.24085774,
       22.75914226])

In [16]:
# eigenequation is satisfied
for i in range(6):
    l = Lt[i]
    v = P[:,i]
    c = M@v - l*v
    print(np.allclose(c, np.zeros(6)))


True
True
True
True
True
True


In [17]:
# Check that M = P Lt P' = P Lt P^-1
np.allclose(P @ np.diag(Lt) @ P.T, M)


True

In [18]:
# Check that Lt = P' M P = P^-1 M P
np.allclose(P.T @ M @ P, np.diag(Lt) )

True

In [19]:
# find max evals and their positions
import operator

def get_topn(L, topn):
    """
    Return the top n values a list L, along with their indices
    """
    index = np.empty(topn, dtype=int) 
    max_evals = np.empty(topn)
    L_c = L.copy()
    for i in range(topn):
        # find highest element of L_c, with its index
        ind, value = max(enumerate(L_c), key=operator.itemgetter(1)) 
        index[i] = ind
        max_evals[i] = value
        # delete element to repeat process
        L_c = np.delete(L_c,ind)
    return index, max_evals

index, max_evals = get_topn(L, 2)

In [20]:
L[index]

array([22.75914226,  2.24085774])

In [21]:
P[:, index]

array([[-0.19501253,  0.24496801],
       [ 0.46689183,  0.1315528 ],
       [ 0.11814577, -0.86645684],
       [-0.46689183, -0.1315528 ],
       [-0.66190436,  0.11341521],
       [-0.27187929, -0.37652082]])

In [22]:
# check that eigenvectors are orthogonal
v = P[:, index[0]]
u = P[:, index[1]]
np.dot(u,v)

-9.71445146547012e-17

In [23]:
# Check that M = P L P' = P L P^-1
A = P_t.T @ M @ P_t
At = A[:2,:2]
np.allclose(At, np.diag(Lt[index]) )

NameError: name 'P_t' is not defined

In [ ]:
Lt

In [ ]:
# However just the first 2 columns of P do not preserve the properties of P
P_t = np.zeros((6,6))
P_t[:,:2] = P[:, index]

np.allclose(P_t @ np.diag(Lt) @ P_t.T, M, )


In [ ]:
# not even orthogonal matrix
np.allclose(P_t @ P_t.T, np.eye(6))

In [50]:
def get_topn_eig(M, topn):
    eset = eigh(M)
    L = eset[0]
    P = eset[1]
    index, max_evals = get_topn(L, 2)
    
    out = dict()
    out['index'] = index
    out['P'] = P[:, index]
    out['L'] = L[index]
    
    return out

ps = get_topn_eig(M, 2)
    


In [51]:
Pt = ps['P']

In [52]:
Lt = ps['L']

In [53]:
for i in range(2):
    if Pt[0,i]<0:
        Pt[:,i] = -Pt[:,i]


In [54]:
np.allclose( Pt @ np.diag(Lt) @ Pt.T, M)


True